### Associate Rule Mining

In [18]:
#Import necessary libraries, pandas, numpy and mlxtend
import pandas as pd
import numpy as np
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [2]:
# Connect to Olympics Database
import os
import psycopg2
from psycopg2 import OperationalError
from dotenv import load_dotenv
import psycopg2
from psycopg2 import OperationalError
 
# Load environment variables
load_dotenv()

def create_connection():
    connection = None
    try:
        connection = psycopg2.connect(
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            host=os.getenv('DB_HOST'),
            port=os.getenv('DB_PORT'),
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

# Create the connection
conn = create_connection()
 

In [3]:

cur = conn.cursor()
# SQL query to select all data from the fact table
fact_data = pd.read_sql("SELECT * FROM fact_olympicathletes", conn)

dim_athlete = pd.read_sql("SELECT * FROM dimathlete", conn)

dim_athleteCountry = pd.read_sql("SELECT * FROM fact_olympicathletes", conn)

dim_economy = pd.read_sql("SELECT * FROM dimeconomy", conn)

dim_event = pd.read_sql("SELECT * FROM dimevent", conn)

dim_game = pd.read_sql("SELECT * FROM dimgame", conn)

dim_medal = pd.read_sql("SELECT * FROM dimmedal", conn)

dim_year = pd.read_sql("SELECT * FROM dimyear", conn)

# Close the cursor and connection
cur.close()
conn.close()

C:\Users\61470\AppData\Local\Temp\ipykernel_32056\803542868.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact_data = pd.read_sql("SELECT * FROM fact_olympicathletes", conn)
C:\Users\61470\AppData\Local\Temp\ipykernel_32056\803542868.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_athlete = pd.read_sql("SELECT * FROM dimathlete", conn)
C:\Users\61470\AppData\Local\Temp\ipykernel_32056\803542868.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_athleteCountry = pd.read_sql("SELECT * FROM fact_olympicathle

In [36]:
merged_df = fact_data.merge(dim_athlete, on='athlete_id')

merged_df.head()


,record_id,athlete_id,athlete_countrycode,event_id,medal_id,game_id,year_id,economy_id,gold_medals,silver_medals,bronze_medals,athlete_name,gender
0,4,3,NOR ...,0,2,beijing-2022,1,NOR002,0,1,0,Magnus NEDREGOTTEN,Mixed
1,5,4,SWE ...,0,3,beijing-2022,1,SWE003,0,0,1,Almida DE VAL,Mixed
2,6,5,SWE ...,0,3,beijing-2022,1,SWE003,0,0,1,Oskar ERIKSSON,Mixed
3,7,6,GBR ...,1,1,beijing-2022,1,GBR004,1,0,0,Athlete_GOLD_Women_GB,Women
4,8,7,JPN ...,1,2,beijing-2022,1,JPN005,0,1,0,Athlete_SILVER_Women_JP,Women


In [37]:
merged_df = merged_df.merge(dim_event, on='event_id')
merged_df = merged_df.merge(dim_medal, on='medal_id')
merged_df.head()

,record_id,athlete_id,athlete_countrycode,event_id,medal_id,game_id,year_id,economy_id,gold_medals,silver_medals,bronze_medals,athlete_name,gender,title,event_gender,discipline,medal_type
0,4,3,NOR ...,0,2,beijing-2022,1,NOR002,0,1,0,Magnus NEDREGOTTEN,Mixed,Mixed Doubles,Mixed ...,Curling,SILVER
1,5,4,SWE ...,0,3,beijing-2022,1,SWE003,0,0,1,Almida DE VAL,Mixed,Mixed Doubles,Mixed ...,Curling,BRONZE
2,6,5,SWE ...,0,3,beijing-2022,1,SWE003,0,0,1,Oskar ERIKSSON,Mixed,Mixed Doubles,Mixed ...,Curling,BRONZE
3,7,6,GBR ...,1,1,beijing-2022,1,GBR004,1,0,0,Athlete_GOLD_Women_GB,Women,Women,Women ...,Curling,GOLD
4,8,7,JPN ...,1,2,beijing-2022,1,JPN005,0,1,0,Athlete_SILVER_Women_JP,Women,Women,Women ...,Curling,SILVER


In [46]:
subset = merged_df [["athlete_countrycode", "medal_type", "gender", 'discipline','title' ]]

In [48]:
subset.drop_duplicates()

,athlete_countrycode,medal_type,gender,discipline,title
0,NOR ...,SILVER,Mixed,Curling,Mixed Doubles
1,SWE ...,BRONZE,Mixed,Curling,Mixed Doubles
3,GBR ...,GOLD,Women,Curling,Women
4,JPN ...,SILVER,Women,Curling,Women
5,SWE ...,BRONZE,Women,Curling,Women
...,...,...,...,...,...
21692,DNK ...,SILVER,Men,Weightlifting,heavyweight - one hand lift men
21693,GRC ...,BRONZE,Men,Weightlifting,heavyweight - one hand lift men
21694,DNK ...,GOLD,Men,Weightlifting,heavyweight - two hand lift men
21695,GBR ...,SILVER,Men,Weightlifting,heavyweight - two hand lift men


In [49]:
subset.head()

,athlete_countrycode,medal_type,gender,discipline,title
0,NOR ...,SILVER,Mixed,Curling,Mixed Doubles
1,SWE ...,BRONZE,Mixed,Curling,Mixed Doubles
2,SWE ...,BRONZE,Mixed,Curling,Mixed Doubles
3,GBR ...,GOLD,Women,Curling,Women
4,JPN ...,SILVER,Women,Curling,Women


In [50]:
new_df = subset.astype(str)


In [51]:
#TransactionEncoder() was designed to covert lists to array
list = new_df.values.tolist()

#Covert the list to one-hot encoded boolean numpy array. 
#Apriori function allows boolean data type only, such as 1 and 0, or FALSE and TRUE.
te = TransactionEncoder()
array_te = te.fit(list).transform(list)

#Check the array
array_te

#Check the colunms
te.columns_

#Apriori function can handle dataframe only, covert the array to a dataframe
arm_df = pd.DataFrame(array_te, columns = te.columns_)
arm_df

,+ 100kg (heavyweight) men,+ 67 kg women,+ 78kg (heavyweight) women,+ 80 kg men,+ 91kg (super heavyweight) men,+ 95kg (heavyweight) men,+105kg men,+75kg women,- 48 kg - extralightweight women,- 48 kg women,...,volleyball men,volleyball women,water polo men,water polo women,york round (100y - 80y - 60y) men,½-1 Ton Race Two Open,épée individual men,épée individual women,épée team men,épée team women
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21692,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21693,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21694,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21695,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [53]:
#Find the frequent itemsets
frequent_itemsets = apriori(arm_df,min_support=0.2,use_colnames =True)

#Check the length of rules
frequent_itemsets['length']=frequent_itemsets['itemsets'].apply(lambda x: len(x))

#Assume the length is 1 and the min support is >= 0.1
frequent_itemsets[ (frequent_itemsets['length']==1) & 
                  (frequent_itemsets['support']>=0.1)]

,support,itemsets,length
0,0.347006,(BRONZE),1
1,0.327649,(GOLD),1
2,0.649445,(Men),1
3,0.325345,(SILVER),1
4,0.292805,(Women),1


In [54]:
#Assume the min confidence is 0.5
rules_con = association_rules(frequent_itemsets, metric="confidence",min_threshold=0.5)
rules_con

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(BRONZE),(Men),0.347006,0.649445,0.226990,0.654137,1.007226,0.001628,1.013568,0.010986
1,(GOLD),(Men),0.327649,0.649445,0.212149,0.647489,0.996989,-0.000641,0.994453,-0.004472
2,(SILVER),(Men),0.325345,0.649445,0.210306,0.646409,0.995326,-0.000988,0.991414,-0.006913


In [55]:
#Assume the min lift is 1
rules_lift = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

#Based on min confidence (=0.5), 
#output antecedents, consequents, support, confidence and lift.
result_arm = rules_con[['antecedents','consequents','support','confidence','lift']]
result_arm

,antecedents,consequents,support,confidence,lift
0,(BRONZE),(Men),0.226990,0.654137,1.007226
1,(GOLD),(Men),0.212149,0.647489,0.996989
2,(SILVER),(Men),0.210306,0.646409,0.995326


In [64]:
#Find the rules whose confidence >= 0.6
new_result_arm = result_arm[result_arm['confidence']>=0.6]
new_result_arm

,antecedents,consequents,support,confidence,lift
0,(BRONZE),(Men),0.226990,0.654137,1.007226
1,(GOLD),(Men),0.212149,0.647489,0.996989
2,(SILVER),(Men),0.210306,0.646409,0.995326


In [66]:
#Find the rules whose confidence >= 0.7
new_result_arm = result_arm[result_arm['confidence']>=0.7]
new_result_arm # None

,antecedents,consequents,support,confidence,lift


In [67]:
#Find the frequent itemsets
frequent_itemsets = apriori(arm_df,min_support=0.2,use_colnames =True)

#Check the length of rules
frequent_itemsets['length']=frequent_itemsets['itemsets'].apply(lambda x: len(x))

#Assume the length is 2 and the min support is >= 0.1
frequent_itemsets[ (frequent_itemsets['length']==2) & 
                  (frequent_itemsets['support']>=0.1)]

,support,itemsets,length
5,0.226990,"(Men, BRONZE)",2
6,0.212149,"(Men, GOLD)",2
7,0.210306,"(Men, SILVER)",2


In [68]:
#Find the frequent itemsets
frequent_itemsets = apriori(arm_df,min_support=0.2,use_colnames =True)

#Check the length of rules
frequent_itemsets['length']=frequent_itemsets['itemsets'].apply(lambda x: len(x))

#Assume the length is 3 and the min support is >= 0.1
frequent_itemsets[ (frequent_itemsets['length']==3) & 
                  (frequent_itemsets['support']>=0.1)] # None

,support,itemsets,length


In [69]:
#Find the frequent itemsets
frequent_itemsets = apriori(arm_df,min_support=0.2,use_colnames =True)

#Check the length of rules
frequent_itemsets['length']=frequent_itemsets['itemsets'].apply(lambda x: len(x))

#Assume the length is 2 and the min support is >= 0.2
frequent_itemsets[ (frequent_itemsets['length']==3) & 
                  (frequent_itemsets['support']>=0.2)] # None

,support,itemsets,length
